In [2]:
from kan import KAN
import matplotlib.pyplot as plt
import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

X_columns = [
    'Header_Length', 'Protocol Type', 'Duration', 'Rate', 'Srate', 
    # 'Drate',
    # 'fin_flag_number', 'syn_flag_number', 'rst_flag_number', 'psh_flag_number',
    # 'ack_flag_number', 'ece_flag_number', 'cwr_flag_number', 'ack_count',
    # 'syn_count', 'fin_count', 'rst_count', 'HTTP', 'HTTPS', 'DNS', 'Telnet',
    # 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IGMP', 
    'IPv','LLC', 
    'Tot sum', 'Min', 'Max', 'AVG', 'Std', 'Tot size', 'IAT', 'Number',
    'Magnitue', 'Radius', 'Covariance',
    'Variance', 'Weight'
]

Y_columns = ['label_L1']

label_L1_mapping = {"MQTT": 0, "Benign": 1, "Recon": 2, "ARP_Spoofing": 3}
label_L2_mapping = {"MQTT-DDoS-Connect_Flood": 4, "MQTT-DDoS-Publish_Flood": 5, 
                    "MQTT-DoS-Connect_Flood": 6, "MQTT-DoS-Publish_Flood": 7,
                    "MQTT-Malformed_Data": 8, "benign": 9, 
                    "Recon-OS_Scan": 10, "Recon-Port_Scan": 11,
                    "arp_spoofing": 12}


# Read the CSV file
df = pd.read_csv('/home/zyang44/Github/baseline_cicIOT/CIC_IoMT/19classes/filtered_train_s_4_11.csv')

df['label_L1'] = df['label_L1'].map(label_L1_mapping)
df['label_L2'] = df['label_L2'].map(label_L1_mapping)

# 90% as training set and 10% left as test set
train_size = int(len(df) * 0.9)
train_df, test_df = df.iloc[:train_size, :], df.iloc[train_size:, :]

scaler = StandardScaler()
train_X_scaled = scaler.fit_transform(train_df[X_columns])
test_X_scaled = scaler.transform(test_df[X_columns])
train_y = train_df[Y_columns].values.ravel()
test_y = test_df[Y_columns].values.ravel()

# take Y_columns as the label, and transfering to one-hot coded
dataset = {
    'train_input': torch.tensor(train_X_scaled, dtype=torch.float32, device=device),
    'train_label': F.one_hot(torch.tensor(train_y, dtype=torch.long, device=device), num_classes=4),
    'test_input': torch.tensor(test_X_scaled, dtype=torch.float32, device=device),
    'test_label': F.one_hot(torch.tensor(test_y, dtype=torch.long, device=device), num_classes=4)
}
print("Data prepared.",
      f"Train set: {dataset['train_input'].shape, dataset['train_label'].shape}",
      f"Test set: {dataset['test_input'].shape, dataset['test_label'].shape}", sep="\n")

cuda:0
Data prepared.
Train set: (torch.Size([33048, 20]), torch.Size([33048, 4]))
Test set: (torch.Size([3672, 20]), torch.Size([3672, 4]))


In [4]:
# create a KAN
kan = KAN(width=[20, 10, 4], grid=5, k=3, seed=42, device=device)

results = kan.fit(dataset, opt="Adam", steps=100)

train_logits = kan(dataset['train_input'])
test_logits = kan(dataset['test_input'])
print("KAN output:",
      f"Train logits {train_logits.shape}",
      f"Test logits {test_logits.shape}", sep="\n")

checkpoint directory created: ./model
saving model version 0.0


description:   0%|                                                          | 0/100 [00:00<?, ?it/s]

| train_loss: 2.57e-01 | test_loss: 4.93e-01 | reg: 1.20e+03 | : 100%|█| 100/100 [00:03<00:00, 32.84


saving model version 0.1
KAN output:
Train logits torch.Size([33048, 4])
Test logits torch.Size([3672, 4])


In [9]:
import ltn
import ltn.fuzzy_ops
from utils import MLP, LogitsToPredicate, DataLoaderMulti

# define the connectives, quantifiers, and the SatAgg
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
And = ltn.Connective(ltn.fuzzy_ops.AndProd())   # And = ltn.Connective(custom_fuzzy_ops.AndProd())
Or = ltn.Connective(ltn.fuzzy_ops.OrProbSum())
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
Exists = ltn.Quantifier(ltn.fuzzy_ops.AggregPMean(p=2), quantifier="e")
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())
SatAgg = ltn.fuzzy_ops.SatAgg()

# define ltn constants
l_MQTT = ltn.Constant(torch.tensor([1, 0, 0, 0, ]))
l_Benign = ltn.Constant(torch.tensor([0, 1, 0, 0]))
l_Recon = ltn.Constant(torch.tensor([0, 0, 1, 0]))
l_ARP_Spoofing = ltn.Constant(torch.tensor([0, 0, 0, 1]))

In [10]:
mlp = MLP(layer_sizes=(20, 32, 4)).to(device)

P_mlp = ltn.Predicate(LogitsToPredicate(mlp))
x = ltn.Variable("x", dataset['test_input'])

print(mlp(dataset['test_input'])[1])

# P_kan = ltn.Predicate(LogitsToPredicate(kan))

# sat_agg = SatAgg()

tensor([ 0.2294,  0.0687,  0.0212, -0.3057], device='cuda:0',
       grad_fn=<SelectBackward0>)
